<a href="https://colab.research.google.com/github/Rehman047/Question-Answering-System--Pytorch-/blob/main/QA_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import torch

In [3]:
df=pd.read_csv('100_Unique_QA_Dataset.csv')
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [8]:
def tokenize(text):
  text=text.lower()
  text=text.replace('?','')
  text=text.replace("'",'')
  return text.split()

In [9]:
vocab={'UNV':0}
def build_vocab(row):
  text=tokenize(row['question'])+tokenize(row['answer'])
  for token in text:
    if token in vocab:
      continue
    vocab[token]=len(vocab)

In [10]:
df.apply(build_vocab,axis=1)


,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [12]:
len(vocab)

324

In [13]:
def text_to_indices(text,vocab):
  indices=[]
  for word in tokenize(text):
    if word not in vocab:
      word='UNV'
    indices.append(vocab[word])
  return indices

In [14]:
print(text_to_indices('Where is paris in my favorite mango?',vocab))

[0, 2, 7, 19, 0, 0, 321]
